# Практическое задание: "Создание чат-бота"

**Выполнил:** Новиков Павел  
**Группа:** DSPR-73  
**Описание:** Создать чат-бот, который по продуктовому запросу рекомендует товары, по остальным запросам отвечает "болталкой".  
**Данные:** Продуктовые запросы находятся в файле  [ProductsDataset.zip](https://drive.google.com/file/d/17OdOPIH9d-T6d1Rb6F0IQuI7P9xoGWHi/view?usp=drive_link)    
Основой для вопросов и ответов являются данные об ответах и вопросах с соответствующего раздела сайта mail.ru [Otvety.txt](https://disk.yandex.ru/d/ikeJ8Tu2LxPkeA)  
**ВНИМАНИЕ!:** ЗАКОММЕЧЕННЫЕ СТРОКИ КОДА СОДЕРЖАТ ДОЛГОВЫПОЛНЯЮЩИЙСЯ КОД. ДЛЯ ЭКОНОМИИ ВРЕМЕНИ РЕЗУЛЬТАТЫ ВЫПОЛНЕНИЯ КОДА В ДАННЫХ ЯЧЕЙКАХ ЗАГРУЖАЮТСЯ В НАЧАЛЕ РАБОТЫ НОУТБУКА.  
**Задачи:**
*   Совершить предобработку текстов в данных об вопросах-ответах и продуктах:
  *   Обучить **word2vec**
  *   Получить эмбеддинги
  *   Удалить знаки препинания
  *   Сделать лемматизацию
* Сложить в индекс все вопросы при помощи библиотеки **annoy**
* Векторизовать ответы, вектором считать усреднённую сумму **word2vec** слов, которые входят в ответ  
* Реализовать метод, который получит на вход вопрос и найдёт ответ к нему.
* Обучить классификатор: продуктовый запрос vs. всё остальное (продуктовым можно считать запрос, который равен названию или описанию товара).
* Добавить логику поиска похожих товаров по продуктовому запросу. Всю логику завернуть в метод get_answer(). Ответ на продуктовый запрос должен иметь вид "product_id title".




In [ ]:
! pip install annoy --quiet
! pip install stop-words --quiet
! pip install pymorphy2 --quiet
! sudo apt-get install cadaver --quiet

Reading package lists...
Building dependency tree...
Reading state information...
cadaver is already the newest version (0.23.3-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [ ]:
import os
# методы работы со строками
import string
# Алгоритм классификации по методу поиска ближайших соседей(ANN) в Python,
# оптимизированный для использования памяти и загрузки/сохранения на диск.
import annoy
# библиотека для раскодирования файлов
import codecs

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec

# выводит индикатор выполнения итератора
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd

import time

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

from annoy import AnnoyIndex

In [ ]:
#Загрузим файл в Google Colab из Яндекс WebDav
def loadFileFromWebDav(site, filepath, filename, login, password):
  if os.path.isfile(filename):
    return

  netrc = os.environ['HOME'] + "/.netrc"

  if not os.path.isfile(netrc): #Check if .netrc file for WebDav access exists
    #!rm $netrc
    netrcText = "machine " + site + " login " + login + " password " + password
    !echo $netrcText > $netrc
    !chmod 600 $netrc
    !cat $netrc

  result = !cadaver --version #Check if WebDav console client cadaver is installed
  if "command not found" in result:
    !sudo apt-get install cadaver

  result = !cadaver --version
  if not ("command not found" in result): #Create file to get the file from WebDav
    #!rm .cadaverrc
    cadaverrc = "cd " + filepath + "\r\n"
    cadaverrc += "get " + filename + "\r\n"
    cadaverrc += "exit\r\n"

    file1 = open("cadaverrc","w")
    file1.write(cadaverrc)
    file1.close()
    !cat "cadaverrc"

    webdavurl = "https://" + site
    !cadaver -r cadaverrc $webdavurl

    if ".zip" in filename: #Unpack file if it was compressed by zip
      !unzip $filename

In [ ]:
# # загрузим необработанный файл с ответами в GoogleColab
# # !rm .cadaverrc
# !rm Otvety.txt
# loadFileFromWebDav("webdav.yandex.ru", "Colab", "Otvety.txt", "pavelzrenie", "nuagieinlgexdsbi")

In [ ]:
# Выведем содержимое файла Otvety.txt на экран
# f = 0
# with codecs.open("Otvety.txt", "r", "utf-8") as fin:
#       # из исходного файла перебираем строки
#       for line in fin:
#         print(line)
#         f += 1
#         if f == 5:
#           break

In [ ]:
# загрузим файл с описанием товаров
!rm ProductsDataset.zip
!rm ProductsDataset.csv
loadFileFromWebDav("webdav.yandex.ru", "Colab", "ProductsDataset.zip", "pavelzrenie", "nuagieinlgexdsbi")

cd Colab
get ProductsDataset.zip
exit
Progress: [=============================>] 100.0% of 4398055 bytes succeeded.
Connection to `webdav.yandex.ru' closed.
Archive:  ProductsDataset.zip
  inflating: ProductsDataset.csv     


In [ ]:
# Загрузим необходимые для работы модели файлы из GoogleDisk в Colab

# В Colab импортируем все необходимые библиотеки
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Привяжем GoogleDisk в Colab
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Загрузим необходимые файлы в Colab

# предобработанные для дальнейших манипуляций данные файла Otvety.txt
# https://drive.google.com/file/d/1OSBU1WbgVrmuIRF9r3CCMpg01-NljG5C/view?usp=drive_link
download = drive.CreateFile({"id": "1OSBU1WbgVrmuIRF9r3CCMpg01-NljG5C"})
download.GetContentFile("prepared_answers.txt")

# векторизованные слова из файла Otvety.txt
# https://drive.google.com/file/d/1-7F4CnTbLIKyD0xbTGAH0OSkKA-oYFcw/view?usp=sharing
download = drive.CreateFile({"id": "1-7F4CnTbLIKyD0xbTGAH0OSkKA-oYFcw"})
download.GetContentFile("model_w2v")

# индексированные вопросы для чата-болталки
# https://drive.google.com/file/d/1St82kpoA9XLvQiiCahoPNVzovgIsdD63/view?usp=sharing
download = drive.CreateFile({"id": "1St82kpoA9XLvQiiCahoPNVzovgIsdD63"})
download.GetContentFile("speaker.ann")

# индексированные ответы для чата-болталки
# https://drive.google.com/file/d/1-AFmyRO_dg1GtoByCgTP1_2NZfrDTOv9/view?usp=sharing
download = drive.CreateFile({"id": "1-AFmyRO_dg1GtoByCgTP1_2NZfrDTOv9"})
download.GetContentFile("index_map")

# данные для продуктового чат-бота ProductsDataset.zip
# https://drive.google.com/file/d/17OdOPIH9d-T6d1Rb6F0IQuI7P9xoGWHi/view?usp=sharing
download = drive.CreateFile({"id": "17OdOPIH9d-T6d1Rb6F0IQuI7P9xoGWHi"})
download.GetContentFile("ProductsDataset.zip")

# данные для создания модели классификации запросов
# https://drive.google.com/file/d/1-7EyqxjKyh7Y4TNBaxQ_mpAUR66TCZeZ/view?usp=drive_link
download = drive.CreateFile({"id": "1-7EyqxjKyh7Y4TNBaxQ_mpAUR66TCZeZ"})
download.GetContentFile("classification_dataset")

# сериализованная модель классификатора
# https://drive.google.com/file/d/1S4GjYmXf9WEtGtt6HUFjTfJGufe5lR3R/view?usp=drive_link
download = drive.CreateFile({"id": "1S4GjYmXf9WEtGtt6HUFjTfJGufe5lR3R"})
download.GetContentFile("model_cs")

In [ ]:
# десериализуем полученные данные
import pickle

with open ('model_w2v', 'rb') as fp:
    model = pickle.load(fp)

with open('index_map','rb') as file:
    index_map = pickle.load(file)

with open ('model_cs', 'rb') as fp:
    model_cs = pickle.load(fp)

with open ('classification_dataset', 'rb') as fp:
    classification_dataset = pickle.load(fp)

In [ ]:
# воссоздадим индескировааные данные о словах для чата-болталки
index = AnnoyIndex(100, 'angular')
index.load('speaker.ann')

True

# Создание функции для препроцессинга текста

In [ ]:
# создадим экземпляр класса для лемматизации
morpher = MorphAnalyzer()
# создадим множество стоп-слов
sw = set(get_stop_words("ru"))
# создадим множество со знаками пунктуации
exclude = set(string.punctuation)

def preprocess_txt(line):
    """Функция для предобработки текста

    Args:
        line (string): обрабатываемый текст

    Returns:
        string: Очищенный от знаков препинания и стоп-слов текст, лемматизированный
                и приведенный к нормальной форме
    """
    # очистка от знаков препинания
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    # принимаем слово (обязательно в нижнем регистре) и возвращаем нормальную форму
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    # удаляем стоп-слова
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

# Данные "болталки"

## Предобработаем данные болталки

In [ ]:
# # вопрос не написан
# question = None
# # ответ не написан
# written = False

# # записываем в файл подготовленные ответы(prepared_answers.txt)
# with codecs.open("prepared_answers.txt","w", "utf-8") as fout:
#   # читаем файл Otvety.txt
#     with codecs.open("Otvety.txt", "r", "utf-8") as fin:
#       # из исходного файла перебираем строки
#         for line in tqdm(fin):
#           # если строка начинается с ---
#             if line.startswith("---"):
#                 # ответ не написан
#                 written = False
#                 # переходим к следующей строке
#                 continue
#             # если строка не начинается с "---", ответ не написан
#             # и у нас есть ответ на вопрос
#             if not written and question is not None:
#                 # записываем строку в файл prepared_answers.txt текст: вопрос+tab+ответ
#                 fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
#                 # ответ написан
#                 written = True
#                 # вопрос не написан
#                 question = None
#                 # идем к следующей строке
#                 continue
#             # если строка не --- , ответ не написан
#             if not written:
#                 # значит это вопрос
#                 question = line.strip()
#                 # переходим к следующей строке
#                 continue
# # сохраним файл для дальнейшего использования
# # index.save('prepared_answers.txt')

## Векторизация слов в данных "болталки" по методу Word2Vec

In [ ]:
# # создадим список предложений
# sentences = []
# # создадим экземпляр класса для лемматизации
# morpher = MorphAnalyzer()
# # создадим множество стоп-слов
# sw = set(get_stop_words("ru"))
# # создадим множество со знаками пунктуации
# exclude = set(string.punctuation)
# # счетчик количества обрабатываемых пар вопрос-ответ
# c = 0

# # открываем файл с необработанными данными "болталки"
# with codecs.open("Otvety.txt", "r", "utf-8") as fin:
#     # используем визуализацию процесса итерации строк
#     for line in tqdm(fin):
#       # преодобработаем тексты вопросов-ответов
#         spls = preprocess_txt(line)
#         # запишем обработанные строки в список
#         sentences.append(spls)
#         c += 1
#         # ограничим количество предобработанных строк до 500000
#         if c > 500000:
#             break

# # Обучим модель word2vec на данных "болталки" для получения векторных представлений слов
# # оставим в списке предложений те, в которых более 2 символов
# sentences = [i for i in sentences if len(i) > 2]
# # размер векторов слов = 100,
# # игнорировать слова с частотой менее 1,
# # максимальное расстояние между текущим и предсказанным словом в предложении = 5
# model = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5)

## Создание индекса вопросов "болталки"

Теперь нам нужно сложить в индекс все вопросы.  
Для создания индекса используем библиотеку **annoy**.   
Проходимся по всем вопросам, здесь нам пригодится предобработанный файл и модель word2Vec.   
Для создания индекса считаем, что вектор предложения - сумма word2vecов слов, которые входят в него (усредненная).

In [ ]:
# # Количество признаков (размерностей) хранимого векторa = 100
# # Метрика расстояния между векторами («угловой»)
# index = annoy.AnnoyIndex(100 ,'angular')

# # словарь, в котором будет находится индекс
# index_map = {}
# # счетчик элементов индекса
# counter = 0

# # для создания индекса используем предобработанный в формат вопрос-ответ исходный файл
# with codecs.open("prepared_answers.txt", "r", "utf-8") as f:
#     # проходим по строкам предобработанного файла
#     for line in tqdm(f):
#       # счетчик количества слов с вектором в вопросе
#         n_w2v = 0
#         # делим строку предобработанного файла на вопрос
#         # и ответ по символу табуляции, проставленному при обработке
#         spls = line.split("\t")
#         # по очереди заполняем словарь(номер индекса-ответ на вопрос)
#         index_map[counter] = spls[1]
#         # производим предобработку вопроса
#         question = preprocess_txt(spls[0])
#         # создаем нулевую матрицу(100,1) из 100 элементов
#         vector = np.zeros(100)
#         # для каждого слова в предобработанном вопросе
#         for word in question:
#           # если даннное слово присутствует в списке созданных векторов
#             if word in model.wv:
#               # суммируем полученные вектора слов
#                 vector += model.wv[word]
#               # увеличиваем счетчик количества слов в вопросе
#                 n_w2v += 1
#       # если в вопросе есть слова
#         if n_w2v > 0:
#           # вычисляем средний вектор
#             vector = vector / n_w2v
#       # записываем в индекс номер индекса и средний вектор входящих в
#       # соответсвующий данному номеру вопрос слов
#         index.add_item(counter, vector)
#       # увеличиваем номер индекса
#         counter += 1
# # количество деревьев в лесу(чем больше, тем точнее)
# index.build(10)
# # сохраним полученный индекс в файл для дальнейшего использования
# # index.save('speaker.ann')

## Поиск ответа в случае, если запрос классифицирован как вопрос "болталки"

In [ ]:
def find_answer_ovety(question):

   """Функция реализует метод, который получает на вход вопрос и находит ответ к нему.
      Для этого происходит препроцессинг вопроса, далее находится ближайший вопрос и
      происходит выбор ответа на ближайший вопрос.

    Args:
        question (string): заданный вопрос

    Returns:
        string: найденный ответ
    """
    # готовим для обработки с помощью препроцессинга заданный вопрос
   preprocessed_question = preprocess_txt(question)
    # счетчик слов в вопросе
   n_w2v = 0
    # создаем вектор
   vector = np.zeros(100)
    # определяем сумму векторов слов входящих в вопрос
    # и количество слов входящих в вопрос
   for word in preprocessed_question:
       if word in model.wv:
          vector += model.wv[word]
          n_w2v += 1
  # определяем средний вектор соответствующий заданному вопросу
   if n_w2v > 0:
      vector = vector / n_w2v
  # ищем 1 ближайший вектор заданного вопроса и вопроса из индекса
   answer_index = index.get_nns_by_vector(vector, 1)
    # возвращаем ответ из наиболее близкого заданному вопроса
   return index_map[answer_index[0]]

In [ ]:
assert(not find_answer_ovety('Где ключи от танка').startswith('5'))
gg = find_answer_ovety('Где ключи от танка')
print(gg)

танки онлайн, мир танков. 



# Данные о продуктах

## Предобработаем данные о продуктах

In [ ]:
# подготовим данные с запросами продуктового чата
products = pd.read_csv('/content/ProductsDataset.zip')
products = products[['title','product_id']]
products.dropna(inplace = True)
products.reset_index(drop= True, inplace = True)
products['content_type'] = np.ones(len(products), dtype = int)
# препроцессинг текста
products['preprocess_txt'] = products['title'].apply(lambda x: preprocess_txt(x))

## Векторизируем слова по методу Word2Vec в данных о продуктах



In [ ]:
## Обучим модель word2vec на названиях продуктов
# оставим в списке предложений те, в которых более 2 символов
sentences = products['preprocess_txt'].tolist()
# создадим модель для получения векторных представлений слов
# размер векторов слов = 30,
# игнорировать слова с частотой менее 1,
# максимальное расстояние между текущим и предсказанным словом в предложении = 3
model_product = Word2Vec(sentences=sentences, vector_size=10, min_count=1, window=5)

## Создание индекса вопросов данных о продуктах

In [ ]:
# Количество признаков (размерностей) хранимого векторa = 30
# Метрика расстояния между векторами («угловой»)
index_product = annoy.AnnoyIndex(10 ,'angular')

# словарь, в котором будет находится индекс
index_product_map = {}
# счетчик элементов индекса
counter = 0

# для создания индекса используем предобработанный в формат вопрос-ответ исходный файл
while counter < len(products):
    # проходим по строкам предобработанного файла
  # счетчик количества слов с вектором в вопросе
  n_w2v = 0
        # по очереди заполняем словарь(номер индекса-идентификационный номер продукта)
  index_product_map[counter] = products.loc[counter,'product_id']
        # создаем нулевую матрицу(30,1) из 30 элементов
  vector = np.zeros(10)
        # для каждого слова в предобработанном вопросе
  for word in products.loc[counter,'preprocess_txt']:
    # если даннное слово присутствует в списке созданных векторов
    if word in model_product.wv:
      # суммируем полученные вектора слов
      vector += model_product.wv[word]
      # увеличиваем счетчик количества слов в вопросе
      n_w2v += 1
      # если в вопросе есть слова
  if n_w2v > 0:
        # вычисляем средний вектор
    vector = vector / n_w2v
      # записываем в индекс номер и средний вектор
  index_product.add_item(counter, vector)
      # увеличиваем счетчик индекса
  counter += 1
# количество деревьев в лесу
index_product.build(30)

# # сохраним полученные данные в файлы для дальнейшего использования
# with open('index_product_map', 'wb') as fp:
#     pickle.dump(index_product_map, fp)

# index_product.save('index_product.ann')

True

## Поиск ответа в случае, если запрос классифицирован как вопрос о продукте

In [ ]:
def find_answer_products(question):
   """Функция реализует метод, который получает на вход вопрос и находит ответ к нему.
      Для этого вопрос  препроцессится, далее находится ближайший вопрос и
      происходит выбор ответа на ближайший вопрос.

    Args:
        question (string): заданный вопрос

    Returns:
        string: найденный ответ
    """
    # готовим для обработки с помощью препроцессинга заданный вопрос
   preprocessed_question = preprocess_txt(question)
  #  print(preprocessed_question)
    # счетчик слов в вопросе
   n_w2v = 0
    # создаем вектор
   vector = np.zeros(10)
    # определяем сумму векторов слов входящих в вопрос
    # и количество слов входящих в вопрос
   for word in preprocessed_question:
    # print(word)
    if word in model_product.wv:
      vector += model_product.wv[word]
      n_w2v += 1
    # определяем средний вектор соответствующий заданному вопросу
   if n_w2v > 0:
    vector = vector / n_w2v
    # ищем 1 ближайший вектор заданного вопроса и вопроса из индекса
    answer_index = index_product.get_nns_by_vector(vector, 5, include_distances=False)
   # возвращаем ответ из наиболее близкого заданному вопроса
   title = products['title'].loc[answer_index[0]]
   return f'{index_product_map[answer_index[0]]} {title}'

In [ ]:
assert(find_answer_products('Юбка детская ORBY').startswith('58e3cfe6132ca50e053f5f82'))
ff = find_answer_products('Юбка детская ORBY')
print(ff)

58e3cfe6132ca50e053f5f82 Юбка детская ORBY


# Создание классификатора запросов чат-бота

In [ ]:
# # создадим файл для обучения модели классификации запросов

# # подготовим данные с запросами вопросов чата-болталки
# dict_questions = {}
# with codecs.open("prepared_answers.txt", "r", "utf-8") as fin:
#       # из исходного файла перебираем строки
#       for line in fin:
#         dict_questions[line.split("\t")[0]] = 0

# content_dataset = pd.DataFrame(list(dict_questions.items()), columns = ['title','content_type'])
# content_dataset['preprocess_txt'] = content_dataset['title'][0:36000].apply(lambda x: preprocess_txt(x))
# content_dataset.dropna(inplace = True)

# # объединение данных в файл для обучения модели
# classification_dataset = pd.concat([content_dataset, products[['title','content_type','preprocess_txt']]])

# # сохранение полученного файла на GoogleDisk
# # with open('classification_dataset', 'wb') as fp:
# #     pickle.dump(classification_dataset, fp)
# # !cp /content/classification_dataset /content/drive/MyDrive/Colab/Векторизация\ и\ классификация\ текста

Данные для создания классификатора подготовлены

После подготовки данных для обучения модели классификации запросов проверим данные на сбалансированность классов и разделим выборку на тренировочную и валидационную части

In [ ]:
# проверка на сбалансированность классов
print(round(classification_dataset['content_type'].value_counts(normalize = True)*100,2))

0    51.77
1    48.23
Name: content_type, dtype: float64


Вывод: Данные сбалансированы

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(classification_dataset['title'],
                                                            classification_dataset['content_type'],
                                                            test_size=0.3,
                                                            random_state=42
                                                            )
# print('Train:\n', y_train.value_counts(normalize=True), sep='')
# print('Valid:\n', y_valid.value_counts(normalize=True), sep='')

# создадим и обучим объект векторизации TfidfVectorizer
vectorizer  = TfidfVectorizer()
values = vectorizer.fit(X_train)

# конвертируем тексты(обучающий и тестовый) в векторы tf-idf
X_train = values.transform(X_train)
X_valid = values.transform(X_valid)

# СОГЛАСНО УСЛОВИЯМ ЗАДАНИЯ МОДЕЛЬ ДОЛЖНА ЗАГРУЖАТЬСЯ ИЗ pkl-файла,
# ЭТО ПРОИСХОДИТ В НАЧАЛЕ РАБОТЫ НОУТБУКА
# # создадим экземпляр класса LogisticRegression()
# model_cs = LogisticRegression()
# # обучаем модель на тренировочной выборке
# model_cs.fit(X_train, y_train)
# делаем предсказание для валидационной выборки
y_valid_pred = model_cs.predict(X_valid)
# выводим значения метрик
print(metrics.classification_report(y_valid, y_valid_pred))

def classification(request):
  """Функция реализует классификацию запросов на продуктовые и "болталку"

    Args:
        request (str): Запрос

    Returns:
        int: 1 если продуктовый запрос, 0 если "болталка"
    """
  vec = vectorizer.transform([request])
  if model_cs.predict(vec)[0] == 1:
    return 1
  else: 0

# # Сериализация модели и перенос в GoogleDrive
# import pickle
# with open('model_cs', 'wb') as fp:
#     pickle.dump(model_cs, fp)
# !cp /content/model_cs /content/drive/MyDrive/Colab/Векторизация\ и\ классификация\ текста

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     10740
           1       1.00      0.98      0.99     10121

    accuracy                           0.99     20861
   macro avg       0.99      0.99      0.99     20861
weighted avg       0.99      0.99      0.99     20861



In [ ]:
# # Сериализация модели и перенос в GoogleDrive
# import pickle
# with open('model_cs', 'wb') as fp:
#     pickle.dump(model_cs, fp)
# !cp /content/model_cs /content/drive/MyDrive/Colab/Векторизация\ и\ классификация\ текста

Вывод: метрики показывающие качество классификации на достаточно высоком уровне, оптимизация не требуется.

In [ ]:
# проверка
requests = ['Где живет кот', 'Ботильоны', 'Юбка детская ORBY', 'Где ключи от танка']
for request in requests:
  response = 'Product' if classification(request) == 1 else 'Question'
  print(f'Запрос: {request}. Ответ чат-бота: {response}')

Запрос: Где живет кот. Ответ чат-бота: Question
Запрос: Ботильоны. Ответ чат-бота: Product
Запрос: Юбка детская ORBY. Ответ чат-бота: Product
Запрос: Где ключи от танка. Ответ чат-бота: Question


# Создание чат-бота

In [ ]:
def get_answer(request):
  """Функция возвращает ответ на вопрос в зависимости от его
     типа(продуктовый или "болталка")

    Args:
        request (str): Вопрос чат-боту

    Returns:
        str: Ответ чат-бота
    """
  if classification(request) == 1:
    return find_answer_products(request)
  return find_answer_ovety(request)

In [ ]:
# проверка
requests = ['Где живет кот', 'Ботильоны', 'Юбка детская ORBY', 'Где ключи от танка']
for request in requests:
  # response = 'Product' if classification(request) == 1 else 'Question'
  print(f'Запрос: {request}. Ответ чат-бота: {get_answer(request)}')

Запрос: Где живет кот. Ответ чат-бота: у Сидоровой козы.... 

Запрос: Ботильоны. Ответ чат-бота: 5667531b2b7f8d127d838c34 Ботильоны
Запрос: Юбка детская ORBY. Ответ чат-бота: 58e3cfe6132ca50e053f5f82 Юбка детская ORBY
Запрос: Где ключи от танка. Ответ чат-бота: танки онлайн, мир танков. 



In [ ]:
# Проверка
assert(get_answer('Юбка детская ORBY').startswith('58e3cfe6132ca50e053f5f82'))
print('True')
assert(not get_answer('Где ключи от танка').startswith('5'))
print('True')

True
True


Результат: Чат-бот классифицирует запросы и выводит релевантные ответы.